low: 1.0 from 0% up to 20%, linear decline up to 40%  
moderate: linear increase from 30% to 40%, 1.0 from 40% to 60%, decline from 60% to 70%  
high:  increase from 0 at 60% to 80%, plateau at 80% to 100%

Compare rise and fall using minimum instead of switchcase/nested if/else.
NumPy operations like (x - a)/(b - a), np.minimum, and np.clip are implemented in compiled C under the hood.
They operate on the entire array at once without Python loops or conditionals.

In [16]:
# --------- Helper function for trapezoidal membership ---------
def trapezoid(x, a, b, c, d):
    """
    Vectorized trapezoidal membership function.
    x: scalar or numpy array
    a, b: start and top-left of trapezoid
    c, d: top-right and end of trapezoid
    Returns same shape as x.
    """
    x = np.asarray(x, dtype=float)

    # Rising edge
    rise = np.zeros_like(x)
    if b != a:
        rise = (x - a) / (b - a)
    else:
        rise[x >= b] = 1.0  # vertical rise, there is no rise

    # Falling edge
    fall = np.zeros_like(x)
    if d != c:
        fall = (d - x) / (d - c)
    else: 
        fall[x <= c] = 1.0  # vertical fall, there is no fall

    # Compute membership: min of rising, plateau=1, falling. 
    # Elements outisde rise will be 1.0, which means it will choose fall value if less than 1, or keep 1 if plateau.
    membership = np.minimum(np.minimum(rise, 1.0), fall)

    # Clip to [0,1]
    return np.clip(membership, 0.0, 1.0)


In [17]:
# --------- Annual Rainfall Layer ------------
# --------- 2-2-1_PH_AR-ANN-RF-1986-2005_MO_2023
# --------- Read raster ---------
def fuzzy_AR():
    raster_path = rf"Component_Layers\AR-86-05-Marikina_WS.tif"
    layer = "AR"
    print(f"Fuzzy rasters for {layer} starting")

    with rasterio.open(raster_path) as src:
        raster = src.read(1).astype(float)  # Read first band
        profile = src.profile

    # --------- Normalize raster to 0-1 ---------
    nodata = -9999
    raster = np.where(raster == nodata, np.nan, raster)

    raster_min = np.nanmin(raster)
    raster_max = np.nanmax(raster)
    print(f"raster_min = {raster_min}")
    print(f"raster_max = {raster_max}")

    # --------- Normalize raster ---------
    denom = raster_max - raster_min
    if denom == 0:
        print("WARNING: raster_min == raster_max — normalization forced to 0")
        raster_norm = np.zeros_like(raster, dtype=float)
    else:
        raster_norm = (raster - raster_min) / denom
    
    # --------- Compute fuzzy memberships ---------
    low_AR = trapezoid(raster_norm, 0.0, 0.0, 0.2, 0.4)
    moderate_AR = trapezoid(raster_norm, 0.2, 0.4, 0.6, 0.8)
    high_AR = trapezoid(raster_norm, 0.6, 0.8, 1.0, 1.0)


    # --------- Save fuzzy rasters ---------
    for layer, name, data in zip([layer]*3, ["low", "moderate", "high"], [low_AR, moderate_AR, high_AR]):
        out_path = rf"Fuzzy_Layers\{layer}_fuzzy_{name}.tif"
        profile.update(dtype=rasterio.float32, count=1)
        with rasterio.open(out_path, "w", **profile) as dst:
            dst.write(data.astype(rasterio.float32), 1)
        print(f"Fuzzy raster saved: {out_path}")

    print(f"Fuzzy rasters for {layer} completed")
    return low_AR, moderate_AR, high_AR

In [18]:
# --------- Elevation Layer ------------
# --------- COG_SRTM90m_CGIAR-CSI_2018 = elevation-upper-marikina-watershed-SRTM.
# --------- DEM_ifsar_30m_phil.tif = NAMRIA 2013
# --------- Read raster ---------
def fuzzy_Elev():
    raster_path = rf"Component_Layers\elevation-upper-marikina-watershed-DEM.tif"
    layer = "elev"
    print(f"Fuzzy rasters for {layer} starting")
    
    with rasterio.open(raster_path) as src:
        raster = src.read(1).astype(float)  # Read first band
        profile = src.profile

    # --------- Normalize raster to 0-1 ---------
    nodata = -9999
    raster = np.where(raster == nodata, np.nan, raster)

    raster_min = np.nanmin(raster)
    raster_max = np.nanmax(raster)
    print(f"raster_min = {raster_min}")
    print(f"raster_max = {raster_max}")

    # --------- Normalize raster ---------
    denom = raster_max - raster_min
    if denom == 0:
        print("WARNING: raster_min == raster_max — normalization forced to 0")
        raster_norm = np.zeros_like(raster, dtype=float)
    else:
        raster_norm = (raster - raster_min) / denom

    # Replace nan with a placeholder (optional)
    #raster_to_save = np.where(np.isnan(raster_norm), -9999, raster_norm)

    # Save as text
    #np.savetxt("normalized_raster.txt", raster_to_save, fmt="%.4f")
    
    # --------- Compute fuzzy memberships ---------
    low_elev = trapezoid(raster_norm, 0.0, 0.0, 0.2, 0.4)
    moderate_elev = trapezoid(raster_norm, 0.2, 0.4, 0.6, 0.8)
    high_elev = trapezoid(raster_norm, 0.6, 0.8, 1.0, 1.0)


    # --------- Save fuzzy rasters ---------
    for layer, name, data in zip([layer]*3, ["low", "moderate", "high"], [low_elev, moderate_elev, high_elev]):
        out_path = rf"Fuzzy_Layers\{layer}_fuzzy_{name}.tif"
        profile.update(dtype=rasterio.float32, count=1)
        with rasterio.open(out_path, "w", **profile) as dst:
            dst.write(data.astype(rasterio.float32), 1)
        print(f"Fuzzy raster saved: {out_path}")

    print(f"Fuzzy rasters for {layer} completed")
    return low_elev, moderate_elev, high_elev, profile


In [19]:
import numpy as np
import rasterio
import os

npver = np.__version__
rasteriover = rasterio.__version__

print(f"Numpy Version = {npver}") #Numpy Version = 2.3.5
print(f"Rasterio Version = {rasteriover}") #Rasterio Version = 1.4.3

low_AR, mod_AR, high_AR = fuzzy_AR()
low_elev, mod_elev, high_elev, profile = fuzzy_Elev()


Numpy Version = 2.3.5
Rasterio Version = 1.4.3
Fuzzy rasters for AR starting
raster_min = 30.79234504699707
raster_max = 37.75746536254883
Fuzzy raster saved: Fuzzy_Layers\AR_fuzzy_low.tif
Fuzzy raster saved: Fuzzy_Layers\AR_fuzzy_moderate.tif
Fuzzy raster saved: Fuzzy_Layers\AR_fuzzy_high.tif
Fuzzy rasters for AR completed
Fuzzy rasters for elev starting
raster_min = 45.785526275634766
raster_max = 1399.4171142578125
Fuzzy raster saved: Fuzzy_Layers\elev_fuzzy_low.tif
Fuzzy raster saved: Fuzzy_Layers\elev_fuzzy_moderate.tif
Fuzzy raster saved: Fuzzy_Layers\elev_fuzzy_high.tif
Fuzzy rasters for elev completed


Fuzzy Gamma Operator


In [20]:
import numpy as np
# Example: two fuzzy membership rasters
# elev_high = ... (numpy array 0-1)
# rain_high = ... (numpy array 0-1)
# USE FUZZY GAMMA OPERATOR

gamma = 0.5  # fuzzy gamma parameter

# dictionary of fuzzy inputs
inputs = {
    "low":  (low_elev,  low_AR),
    "mod":  (mod_elev,  mod_AR),
    "high": (high_elev, high_AR)
}

fuzzy_gamma_outputs = {}

for cls, (elev_fuzzy, AR_fuzzy) in inputs.items():

    # Fuzzy AND (algebraic product)
    fuzzy_and = elev_fuzzy * AR_fuzzy

    # Fuzzy OR (algebraic sum)
    fuzzy_or = elev_fuzzy + AR_fuzzy - (elev_fuzzy * AR_fuzzy)

    # Fuzzy Gamma
    suit_fuzzy = (fuzzy_and ** (1 - gamma)) * (fuzzy_or ** gamma)

    # clip 0–1 for safety
    suit_fuzzy = np.clip(suit_fuzzy, 0, 1)

    # store result
    fuzzy_gamma_outputs[cls] = suit_fuzzy

    print(f"Computed fuzzy gamma suitability for class: {cls}")

Computed fuzzy gamma suitability for class: low
Computed fuzzy gamma suitability for class: mod
Computed fuzzy gamma suitability for class: high


The centroid method (also called fuzzy weighted average) converts these multiple fuzzy rasters into a single continuous suitability map. (low = 1/3, mod = 2/3, high=1)

In [21]:
import numpy as np

# Example fuzzy rasters dictionary
# fuzzy_gamma_outputs = {"low": low_array, "mod": mod_array, "high": high_array}
# Weights for each class (can all be 1)
class_weights = {"low": 1, "mod": 1, "high": 1}

# Get raster shape
shape = next(iter(fuzzy_gamma_outputs.values())).shape

# Initialize numerator and denominator arrays
numerator = np.zeros(shape, dtype=float)
denominator = np.zeros(shape, dtype=float)

# Compute weighted average per pixel
for cls, weight in class_weights.items():
    fuzzy_raster = fuzzy_gamma_outputs[cls]
    numerator += fuzzy_raster * weight
    denominator += fuzzy_raster

# Avoid division by zero
final_suitability = np.where(denominator == 0, 0, numerator / denominator)

# Clip to [0,1] to be safe
final_suitability = np.clip(final_suitability, 0, 1)

out_path = r"Fuzzy_Layers\final_suitability.tif"
profile.update(dtype=rasterio.float32, count=1)
with rasterio.open(out_path, "w", **profile) as dst:
    dst.write(final_suitability.astype(rasterio.float32), 1)
    
print("Final suitability raster computed per pixel and saved.")

Final suitability raster computed per pixel and saved.


C:\Users\Darwin\AppData\Local\Temp\ipykernel_1056\1481216898.py:22: RuntimeWarning: invalid value encountered in divide
  final_suitability = np.where(denominator == 0, 0, numerator / denominator)
